# Section 1-0 - Prelude

We'll start with the Kaggle Titanic dataset. The dataset is a list of Titanic passengers with features such as age, sex and class. We'll use these features to train a model, and use the model to predict whether or not each passenger survived. A more detailed treatment of the dataset can be found here:

https://github.com/savarin/python_for_ml

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)

df = pd.read_csv('data/titanic.csv')

In [2]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


We split the data 80-20 into training and test sets, fill in missing values and convert strings to numbers. In addition, we scale the data so that each column has mean 0 and standard deviation 1, and create one-hot vectors with the labels (analogous to dummy variables).

In [3]:
df_train = df.iloc[:712, :]

df_train = df_train.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

age_mean = df_train['Age'].mean()
df_train['Age'] = df_train['Age'].fillna(age_mean)
df_train['Sex'] = df_train['Sex'].map({'female': 0, 'male': 1}).astype(int)

features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
scaler = StandardScaler()

X_train = scaler.fit_transform(df_train[features].values)
y_train = df_train['Survived'].values
y_train_onehot = pd.get_dummies(df_train['Survived']).values

In [4]:
df_test = df.iloc[712:, :]

df_test = df_test.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

df_test['Age'] = df_test['Age'].fillna(age_mean)
df_test['Sex'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)

X_test = scaler.transform(df_test[features].values)
y_test = df_test['Survived'].values

 ## Benchmark

To create a basis for comparison, we train a Random Forest model and record the accuracy on the test set.

In [5]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0, verbose=3)
model = model.fit(X_train, y_train)

y_prediction = model.predict(X_test)
print "\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test))

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10

accuracy 0.832402234637


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


## 1-layer Neural Network

For the input, we have a vector of length 6 that represents each passenger's features. As an example, we consider the first passenger in the dataset.

In [6]:
print X_train[0]

[ 0.83290956  0.74926865 -0.6178933   0.4436936  -0.47015218 -0.51933199]


For the output, we want a vector of length 2 to represent the survival probabilities. A simple way to create this mapping is by using a 2x6 matrix. To start off, we generate a random matrix representing feature weights and apply the matrix to our input. We'll also add a bias term.

In [7]:
W = np.random.rand(2, 6) / 10

print W

[[ 0.02620247  0.0158684   0.02781265  0.04593169  0.03210005  0.05183928]
 [ 0.02619429  0.09760853  0.07328146  0.01152742  0.03862751  0.06285012]]


In [8]:
b = np.random.rand(2,) / 10

print b

[ 0.01250579  0.09835486]


In [9]:
result = np.dot(W, X_train[0]) + b

print result

[ 0.00740041  0.10234099]


To get the output vector to sum to 1, we apply a softmax mapping. The first element would now represent the probability that the passenger did not survive, and the second element represents the probability the passenger survives.

In [10]:
def softmax(x):
    return np.exp(x) / np.exp(x).sum()

In [11]:
result = softmax(result)

print result

[ 0.47628267  0.52371733]


We can then compare the output vector to the actual label. We would have a 'good' model if the probability for the correct label was close to 1, and a 'bad' one if it was close to 0.

In [12]:
print y_train_onehot[0]

[ 1.  0.]


In [13]:
label_index = np.argmax(y_train_onehot[0])

print label_index

0


In [14]:
print result[label_index]

0.476282667565


We define loss to be the negative logarithm of the probability of the correct label. Taking the logarithm penalizes the model for having a high probability associated with the wrong label. Here we have the loss associated with the first passenger.

In [15]:
loss = -np.log(result[label_index])

print loss

0.741743761582


We now iterate through 1000 iterations for random values of W and b, and keep the pair which minimizes the average loss across all passengers.

In [16]:
min_loss = 1000
best_weights = ()

start = time()

for i in xrange(1000):
    W = np.random.rand(2, 6) / 10
    b = np.random.rand(2,) / 10

    scores = []
    loss = 0
    
    for j in xrange(X_train.shape[0]):
        result = np.dot(W, X_train[j]) + b
        result = softmax(result)
        scores.append(list(result))
        
        label_index = np.argmax(y_train_onehot[j])
        loss += -np.log(result[label_index])

    loss = loss / float(X_train.shape[0])
    y_prediction = np.argmax(np.array(scores), axis=1)
    accuracy = np.sum(y_prediction == y_train) / float(len(y_train))
    
    if loss < min_loss:
        min_loss = loss
        best_weights = (W, b)
        print 'loss %s accuracy %s loop %s' % (round(loss, 3), round(accuracy, 3), i)

print '\ntime taken %s seconds' % str(time() - start)

loss 0.685 accuracy 0.64 loop 0
loss 0.683 accuracy 0.642 loop 1
loss 0.681 accuracy 0.695 loop 5
loss 0.675 accuracy 0.712 loop 8
loss 0.673 accuracy 0.737 loop 44
loss 0.661 accuracy 0.739 loop 47
loss 0.658 accuracy 0.715 loop 297
loss 0.652 accuracy 0.77 loop 514

time taken 8.20281195641 seconds


In [17]:
W, b = best_weights
scores = []

for j in xrange(X_test.shape[0]):
    result = np.dot(W, X_test[j]) + b
    result = softmax(result)
    scores.append(list(result))

y_prediction = np.argmax(np.array(scores), axis=1)

print "accuracy", np.sum(y_prediction == y_test) / float(len(y_test))

accuracy 0.810055865922


For each passenger, predictions for the test set were made by selecting the label with the highest probability. Despite the naïve approach, we obtain a prediction accuracy of 81%!

## 2-layer Neural Network

With the 1-layer neural network, we had a 2x6 weight matrix. To create more degrees of freedom, we can introduce intermediary matrices or 'layers'. For example, instead of having a mapping from a vector of length 6 to a vector of length 2, we'll have two mappings - first from 6 to 100, followed by 100 to 2.

In [18]:
W_1 = np.random.rand(100, 6) / 10
b_1 = np.random.rand(100,) / 10
W_2 = np.random.rand(2, 100) / 10
b_2 = np.random.rand(2,) / 10

In [19]:
result = np.dot(W_1, X_train[0]) + b_1

print result

[  3.85833932e-02   5.89968658e-02   5.25942066e-03   7.43856347e-02
   1.14824434e-05   1.15027700e-01   3.68012540e-02   1.09882469e-01
   9.05096555e-02   1.49145180e-01   4.92299995e-02   7.15835553e-02
  -2.30711740e-02   8.81330372e-02   3.01255392e-03   1.08960642e-01
   6.98125076e-02   5.54114438e-02   6.55790909e-02   8.08632209e-02
   1.16684986e-01   1.64563621e-01   4.72968084e-02   6.44781150e-02
   6.56903272e-02   3.71233636e-02   4.19944730e-02   1.65645958e-01
   2.50494467e-02   1.94216742e-02   9.79751463e-02   3.02635378e-02
   6.33219632e-04   8.73328656e-02   2.90520962e-02   9.43040144e-02
   1.01387639e-01   6.37643873e-02   1.33784314e-01   8.22222654e-02
   2.26342486e-01   1.60054641e-01   1.47240435e-01   7.64466972e-02
   8.22117015e-02   8.62467846e-02  -3.58229687e-02   1.11880085e-01
   1.65073281e-02   1.70666967e-01   7.91422152e-02   3.39914112e-02
   1.22783230e-01   1.47883196e-01   1.37466479e-01   1.43118761e-01
   2.83526009e-02   1.20861416e-01

In [20]:
result = np.dot(W_2, result) + b_2

print result

[ 0.46288862  0.44362129]


As before we iterate through 1000 iterations of random values for W_1, b_1, W_2 and b_2, and keep the one which minimizes loss.

In [21]:
min_loss = 1000
best_weights = ()

start = time()

for i in xrange(1000):
    W_1 = np.random.rand(100, 6) / 10
    b_1 = np.random.rand(100,) / 10
    W_2 = np.random.rand(2, 100) / 10
    b_2 = np.random.rand(2,) / 10
    
    scores = []
    loss = 0

    for j in xrange(X_train.shape[0]):
        result = np.dot(W_1, X_train[j]) + b_1
        result = np.dot(W_2, result) + b_2
        result = softmax(result)
        scores.append(list(result))
        
        label_index = np.argmax(y_train_onehot[j])
        loss += -np.log(result[label_index])

    loss = loss / float(X_train.shape[0])
    y_prediction = np.argmax(np.array(scores), axis=1)
    accuracy = np.sum(y_prediction == y_train) / float(len(y_train))
        
    if loss < min_loss:
        min_loss = loss
        best_weights = (W_1, b_1, W_2, b_2)
        print 'loss %s accuracy %s loop %s' % (round(loss, 3), round(accuracy, 3), i)

print '\ntime taken %s seconds' % str(time() - start)

loss 0.682 accuracy 0.61 loop 0
loss 0.679 accuracy 0.757 loop 5
loss 0.678 accuracy 0.657 loop 8
loss 0.675 accuracy 0.787 loop 15
loss 0.67 accuracy 0.676 loop 29
loss 0.668 accuracy 0.633 loop 333
loss 0.663 accuracy 0.702 loop 362
loss 0.663 accuracy 0.708 loop 590

time taken 9.62385892868 seconds


In [22]:
W_1, b_1, W_2, b_2 = best_weights
scores = []

for j in xrange(X_test.shape[0]):
    result = np.dot(W_1, X_test[j]) + b_1
    result = np.dot(W_2, result) + b_2
    result = softmax(result)
    scores.append(list(result))
    
y_prediction = np.argmax(np.array(scores), axis=1)

print "accuracy", np.sum(y_prediction == y_test) / float(len(y_test))

accuracy 0.670391061453


Despite the greater degree of freedom, we get an accuracy score of only 67%.

## 3-layer Neural Network

We take this a step further by adding an additional layer, and similarly review model performance.

In [23]:
min_loss = 1000
best_weights = ()

start = time()

for i in xrange(1000):
    W_1 = np.random.rand(100, 6) / 10
    b_1 = np.random.rand(100,) / 10
    W_2 = np.random.rand(100, 100) / 10
    b_2 = np.random.rand(100,) / 10
    W_3 = np.random.rand(2, 100) / 10
    b_3 = np.random.rand(2,) / 10
    
    scores = []
    loss = 0

    for j in xrange(X_train.shape[0]):
        result = np.dot(W_1, X_train[j]) + b_1
        result = np.dot(W_2, result) + b_2
        result = np.dot(W_3, result) + b_3
        result = softmax(result)
        scores.append(list(result))
        
        label_index = np.argmax(y_train_onehot[j])
        loss += -np.log(result[label_index])
        
    loss = loss / float(X_train.shape[0])
    y_prediction = np.argmax(np.array(scores), axis=1)
    accuracy = np.sum(y_prediction == y_train) / float(len(y_train))          
        
    if loss < min_loss:
        min_loss = loss
        best_weights = (W_1, b_1, W_2, b_2, W_3, b_3)
        print 'loss %s accuracy %s loop %s' % (round(loss, 3), round(accuracy, 3), i)

print '\ntime taken %s seconds' % str(time() - start)

loss 0.797 accuracy 0.287 loop 0
loss 0.753 accuracy 0.296 loop 1
loss 0.752 accuracy 0.285 loop 2
loss 0.695 accuracy 0.39 loop 4
loss 0.656 accuracy 0.705 loop 5
loss 0.644 accuracy 0.709 loop 7
loss 0.642 accuracy 0.705 loop 12
loss 0.633 accuracy 0.709 loop 19
loss 0.621 accuracy 0.721 loop 79
loss 0.621 accuracy 0.715 loop 130
loss 0.619 accuracy 0.728 loop 501
loss 0.618 accuracy 0.715 loop 956

time taken 13.7536799908 seconds


In [24]:
W_1, b_1, W_2, b_2, W_3, b_3 = best_weights
scores = []

for j in xrange(X_test.shape[0]):
    result = np.dot(W_1, X_test[j]) + b_1
    result = np.dot(W_2, result) + b_2
    result = np.dot(W_3, result) + b_3    
    result = softmax(result)
    scores.append(list(result))
    
y_prediction = np.argmax(np.array(scores), axis=1)

print "accuracy", np.sum(y_prediction == y_test) / float(len(y_test))

accuracy 0.703910614525


We run into the same problem of suboptimal model performance, but this is not unexpected given the simplistic approach taken for illustrative purposes. We look to build on this in the next section by taking a systematic approach to optimizing the weight matrices and biases.